In [10]:
#
# Python module import.
#

import sys
sys.path.append("../")
import math
import copy
import pickle

import torch

import matplotlib.pyplot as plt

import imodal

device = 'cpu'
torch.set_default_dtype(torch.float64)
imodal.Utilities.set_compute_backend('keops')

In [2]:
bas = imodal.Models.Deformable3DImage.load_from_file("/home/leander/data/imagen/27113512/regions/nat_amyg_000027113512_bas_roi.nii.gz")
# bas = imodal.Models.Deformable3DImage.load_from_file("/home/leander/data/imagen/27113512/regions/nat_amyg_000027113512_bas.nii")
# fu2 = imodal.Models.Deformable3DImage.load_from_file("/home/leander/data/imagen/27113512/regions/nat_amyg_000027113512_fu2.nii")
fu3 = imodal.Models.Deformable3DImage.load_from_file("/home/leander/data/imagen/27113512/regions/nat_amyg_000027113512_fu3_roi.nii.gz")
# fu3 = imodal.Models.Deformable3DImage.load_from_file("/home/leander/data/imagen/27113512/regions/nat_amyg_000027113512_fu3.nii")


In [3]:
print(bas.bitmap.shape)
# print(fu2.bitmap.shape)
print(fu3.bitmap.shape)

torch.Size([67, 44, 83])
torch.Size([67, 44, 83])


In [5]:
deformables_shape = bas.bitmap.shape

aabb = imodal.Utilities.AABB(0., deformables_shape[0], 0., deformables_shape[1], 0., deformables_shape[2])

# points_density = 0.01
points_density = 0.0001
lddmm_points = aabb.fill_uniform_density(points_density)

In [6]:
print(lddmm_points.shape)
print(lddmm_points.dtype)

torch.Size([48, 3])
torch.float64


In [7]:
scale = 2./points_density**(1/3)
print(scale)
lddmm = imodal.DeformationModules.ImplicitModule0(3, lddmm_points.shape[0], scale, gd=lddmm_points, nu=100.)

43.08869380063767


In [8]:
model = imodal.Models.RegistrationModel(bas, [lddmm], imodal.Attachment.L2NormAttachment())

In [9]:
shoot_solver = 'euler'
shoot_it = 10

costs = {}
fitter = imodal.Models.Fitter(model, optimizer='torch_lbfgs')
fitter.fit([fu3], 100, costs=costs, options={'shoot_solver': shoot_solver, 'shoot_it': shoot_it, 'line_search_fn': 'strong_wolfe'})

Starting optimization with method torch LBFGS
Initial cost={'deformation': tensor(0.), 'attach': tensor(1.7566e+09)}


Evaluated model with costs=1756591003.0


Evaluated model with costs=1687984215.0043733


Evaluated model with costs=1679204489.1673267


Evaluated model with costs=1674137442.9778738


...

Evaluated model with costs=1673823785.4068718


Evaluated model with costs=1673823785.399829


Evaluated model with costs=1673823785.3989954


Evaluated model with costs=1673823785.395977


RuntimeError: python_error